In [2]:
%matplotlib ipympl

import matplotlib.pyplot as plt
import numpy as np
import obspy

In [424]:
st0 = obspy.read('/home/malcolmw/data/wfs/070/ALCY.2013.070.mseed')
st0.filter('bandpass', freqmin=1, freqmax=20)

3 Trace(s) in Stream:
YN.ALCY..EHE | 2013-03-11T00:00:00.004500Z - 2013-03-11T23:59:59.984500Z | 50.0 Hz, 4320000 samples
YN.ALCY..EHN | 2013-03-11T00:00:00.004500Z - 2013-03-11T23:59:59.984500Z | 50.0 Hz, 4320000 samples
YN.ALCY..EHZ | 2013-03-11T00:00:00.004500Z - 2013-03-11T23:59:59.984500Z | 50.0 Hz, 4320000 samples

In [429]:
st = st0.slice(
    starttime=obspy.UTCDateTime('2013-03-11T18:25:30'),
    endtime=obspy.UTCDateTime('2013-03-11T18:26:00')
)
# st.normalize()
st.plot(handle=True);

FigureCanvasNbAgg()

In [430]:
sample_rate = st[0].stats.sampling_rate

trz = st.select(channel='??Z*')[0].data
trn = st.select(channel='??[N1]*')[0].data
tre = st.select(channel='??[E2]*')[0].data

window_length_in_seconds = 1
nsamp = int(window_length_in_seconds * sample_rate)
shape = (trz.size - nsamp + 1)
xx = np.lib.stride_tricks.as_strided(trz, shape=(shape, nsamp), strides=(trz.itemsize, trz.itemsize))
yy = np.lib.stride_tricks.as_strided(trn, shape=(shape, nsamp), strides=(trz.itemsize, trz.itemsize))
zz = np.lib.stride_tricks.as_strided(tre, shape=(shape, nsamp), strides=(trz.itemsize, trz.itemsize))
xyz = np.stack([xx, yy, zz], axis=1)
eigh = [np.linalg.eigh(np.cov(window)) for window in xyz]
eigval = np.stack([np.flip(eigh[idx][0]) for idx in range(len(eigh))])
eigvec = np.stack([np.fliplr(eigh[idx][1]) for idx in range(len(eigh))])
rect = 1 - (eigval[:, 1] + eigval[:, 2]) / (2 * eigval[:, 0])
plan = 1 - (2 * eigval[:, 2]) / (eigval[:, 0] + eigval[:, 1])
azim = np.arctan2((eigvec[:, 0, 1]), (eigvec[:, 0, 2]))
inc = np.arccos(np.abs(eigvec[:, 0, 0]))

In [441]:
plt.close('all')

axes = []
fig = plt.figure(figsize=(5, 5))
s_filt = rect * (1 - np.cos(inc))
p_filt = rect * np.cos(inc)
filt = np.diff(p_filt)
ax = fig.add_subplot(4, 1, 1)
axes.append(ax)
# ax.plot(filt, 'k')
ax.plot(rect, label='Rectilinearity')
ax.plot(plan, label='Planarity')
ax.plot(np.cos(inc), label='cos(incidence)')
ax.set_ylim(0, 1)
ax.legend(loc=7)

row = 1
for tr in (trz, trn, tre):
    row += 1
    ax = fig.add_subplot(4, 1, row)
    axes.append(ax)
    tr = tr[len(tr)-len(filt):]
    tr = tr / np.max(np.abs(tr))
    ax.plot(tr, 'r')
    ax.plot(tr * filt, 'k')

for ax in axes:
    ax.set_xlim(0, len(filt))
for ax in axes[1:]:
    ax.set_ylim(-1, 1)
for ax in axes[:-1]:
    ax.set_xticks([])
axes[-1].set_xticklabels(axes[-1].get_xticks() / sample_rate)

plt.subplots_adjust(hspace=0)

FigureCanvasNbAgg()

In [448]:
plt.close('all')

fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
filt = rect * np.cos(inc)

tr = trz[len(trz)-len(filt):]
ax.plot(tr / np.max(np.abs(tr)), 'k')

param = rect * np.cos(inc)
ax.plot(param)
ax.plot(np.diff(param))

FigureCanvasNbAgg()

In [436]:
np.diff?

Signature: np.diff(a, n=1, axis=-1)
Docstring:
Calculate the n-th discrete difference along the given axis.

The first difference is given by ``out[n] = a[n+1] - a[n]`` along
the given axis, higher differences are calculated by using `diff`
recursively.

Parameters
----------
a : array_like
    Input array
n : int, optional
    The number of times values are differenced. If zero, the input
    is returned as-is.
axis : int, optional
    The axis along which the difference is taken, default is the
    last axis.

Returns
-------
diff : ndarray
    The n-th differences. The shape of the output is the same as `a`
    except along `axis` where the dimension is smaller by `n`. The
    type of the output is the same as the type of the difference
    between any two elements of `a`. This is the same as the type of
    `a` in most cases. A notable exception is `datetime64`, which
    results in a `timedelta64` output array.

See Also
--------
gradient, ediff1d, cumsum

Notes
-----
Type is pres

In [421]:
eigvec.shape

(702, 3, 3)